# Become a movie director

Let's use Scrapy to get some information about the 250 top rated movies on <a href="http://www.imdb.com/" target="_blank">IMDB</a>.

1. Install `Scrapy`:

In [ ]:
# !pip install scrapy

## Étape 1 — Extraire les informations d’un seul film

https://www.imdb.com/fr/chart/boxoffice/

2. Lance une requête vers la page IMDb :

In [60]:
import requests
from scrapy import Selector

In [61]:
url = "https://www.imdb.com/chart/top/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

response = requests.get(url,headers=headers)

print(response.status_code)
print(response.text)

200
<!DOCTYPE html><html lang="fr-FR" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1770916980104);
        }
    })</script><title>Les 250 meilleurs films selon IMDb</title><meta name="description" content="Évalué par les votants réguliers d&#x27;IMDb." data-id="main"/><meta name="google-site-verification" content="0cadf7898134e79b"/><meta name="msvalidate.01" content="C1DACEF2769068C0B0D2

<!DOCTYPE html><html lang="fr-FR" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1770905967749);
        }
    })</script><title>Les 250 meilleurs films selon IMDb</title><meta name="description" content="Évalué par les votants réguliers d&#x27;IMDb." data-id="main"/><meta name="google-site-verification" content="0cadf7898134e79b"/><meta name="msvalidate.01" content="C1DACEF2769068C0B0D2687C

In [62]:
sel = Selector(text=response.text)
sel

<Selector query=None data='<html lang="fr-FR" xmlns:og="http://o...'>

3. Identifie la balise principale contenant les films :

In [63]:
films = sel.css("li.ipc-metadata-list-summary-item")
films

[<Selector query="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' ipc-metadata-list-summary-item ')]" data='<li class="ipc-metadata-list-summary-...'>,
 <Selector query="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' ipc-metadata-list-summary-item ')]" data='<li class="ipc-metadata-list-summary-...'>,
 <Selector query="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' ipc-metadata-list-summary-item ')]" data='<li class="ipc-metadata-list-summary-...'>,
 <Selector query="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' ipc-metadata-list-summary-item ')]" data='<li class="ipc-metadata-list-summary-...'>,
 <Selector query="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' ipc-metadata-list-summary-item ')]" data='<li class="ipc-metadata-list-summary-...'>,
 <Selector query="descendant-or-self::li

4. Sélectionne le premier film, récupère pas à pas :
- le classement
- le titre du film
- la note IMDb
- le nombre de votes (en nettoyant la valeur : '84\xa0k' → 84000)
- les revenus s’ils sont affichés ('34\xa0M\xa0$US' → 34000000)

💡 Utilise `.css("...::text").get()` pour tester les sélecteurs un par un.

💡 Quand un texte est découpé en plusieurs morceaux (['\xa0(', '84\xa0k', ')']), pense à utiliser `.getall()` puis `''.join(...)`.

In [64]:
premier_film = films[0]
print(premier_film)

<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable" aria-disabled="false"></span><div class="sc-fc35a1ef-1 lmHCrT cli-parent li-compact"><div class="sc-fc35a1ef-0 hTMtRz"><div class="sc-d0224b4e-0 jfogmY cli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Morgan Freeman and Tim Robbins in Les Évadés (1994)" class="ipc-image" loading="lazy" src="https://m.media-amazon.com/images/M/MV5BOTc2MjMxNzIyMF5BMl5BanBnXkFtZTgwNDQ2MjEyMDE@._V1_QL75_UY133_CR5,0,90,133_.jpg" srcset="https://m.media

In [65]:
# Titre du film
premier_titre = premier_film.css("h3[class='ipc-title__text']::text").get()
premier_titre

'Les Évadés'

In [66]:
# Url ??? 
tmp_scrap= premier_film.css('a.ipc-title-link-wrapper::attr(href)').get(default='')
premier_url=f"https://www.imdb.com{tmp_scrap}"
print(premier_url)

https://www.imdb.com/fr/title/tt0111161/?ref_=chttp_t_1


In [67]:
#premier titre
premier_titre = premier_film.css('h3.ipc-title__text::text').get()
print(premier_titre)

#premier lien
#on doit requeter une autre page pour les earnings
base_url="https://www.imdb.com"
film_relative_url = premier_film.css('a.ipc-title-link-wrapper::attr(href)').get('')
film_url = base_url+film_relative_url  # → https://www.imdb.com/title/tt0111161/
print(film_url)

#partie financiere sur une autre page
detail_resp = requests.get(film_url, headers=headers, timeout=10)
detail_sel = Selector(text=detail_resp.text)

#on cumule le filtrage avec la balise parente et les classes, puis on cible le bon item avec l'attribut data-testid
gross_worldwide = detail_sel.css(
    'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
    'span.ipc-metadata-list-item__list-content-item::text'
).get(default='?').strip()

#budget sur le meme mode
budget = detail_sel.css(
    'li[data-testid="title-boxoffice-budget"] '
    'span.ipc-metadata-list-item__list-content-item ::text'
).get()

print("Budget :", budget)
print("Gross mondial :", gross_worldwide)

Les Évadés
https://www.imdb.com/fr/title/tt0111161/?ref_=chttp_t_1
Budget : 25 000 000 $US (estimé)
Gross mondial : 29 334 033 $US


In [ ]:
# Earnings
premier_earnings_raw = premier_film.css("").get()
premier_earnings_raw

'9\xa0M\xa0$US'

In [68]:
# Fonction pour nettoyer le string
import re
def parse_earnings(raw_text):
    """Nettoie et convertit un chiffre d'affaires IMDb (ex: '5,3\xa0M\xa0$US')"""
    if not raw_text:
        return None
    text = raw_text.replace('\xa0', ' ').replace('$US', '').strip()
    match = re.search(r'([\d\s.,]+)\s*([KkMm]?)', text)
    if not match:
        return None

    number = match.group(1).replace(' ', '').replace(',', '.')
    suffix = match.group(2).upper()
    try:
        value = float(number)
    except ValueError:
        return None

    if suffix == 'K':
        value *= 1_000
    elif suffix == 'M':
        value *= 1_000_000
    return int(value)

In [11]:
premier_earnings = parse_earnings(premier_earnings_raw)
premier_earnings

9000000

In [69]:
# Le rating
premier_rating = premier_film.css("span.ipc-rating-star--rating::text").get()
premier_rating

'9,3'

In [70]:
# Nombre de voteurs
premier_nb_voteurs = premier_film.css("span.ipc-rating-star--voteCount::text").get()
premier_nb_voteurs

'\xa0('

In [71]:
# Nombre de voteurs
premier_nb_voteurs_raw = premier_film.css("span.ipc-rating-star--voteCount::text").getall()
premier_nb_voteurs_raw

['\xa0(', '3,2\xa0M', ')']

In [72]:
import re

def parse_vote_count(film):
    # Exemple de texte réel que tu obtiens souvent
    texte = "".join(film.css("span.ipc-rating-star--voteCount::text").getall())
    # → typiquement : ' (3,2\xa0M)'   ou ' (1,45\xa0M)'   ou ' (456\xa0K)'   ou ' (12345)'

    # Regex qui capture le nombre ET le suffixe optionnel (M/K)
    match = re.search(r'([\d,.]+)\s*([KM])?', texte, re.IGNORECASE)

    if match:
        nombre_str = match.group(1)          # '3,2' ou '1,45' ou '456'
        suffixe    = match.group(2) or ''    # 'M' ou 'K' ou ''

        # Nettoyage
        nombre_str = nombre_str.replace(',', '.').strip()   # '3.2'

        # Multiplicateur
        multi = 1
        if suffixe.upper() == 'M':
            multi = 1_000_000
        elif suffixe.upper() == 'K':
            multi = 1_000

        try:
            votes = int(float(nombre_str) * multi)
            print(votes)                    # → 3200000
            print(f"{votes:,}")             # → 3,200,000 (plus joli)
        except ValueError:
            print("Conversion impossible")
    else:
        print("Aucun nombre trouvé")            # → 320000

parse_vote_count(premier_film)


3200000
3,200,000


In [73]:
premier_nb_voteurs = parse_vote_count(premier_film)
premier_nb_voteurs

3200000
3,200,000


## Étape 2 — Extraire les informations de tous les films
Une fois que ton code fonctionne pour un film, généralise-le à toute la liste.

### 1. Boucle sur chaque élément de films.
### 2. Applique les mêmes sélecteurs et nettoyages.
### 3. Stocke les résultats dans une liste de dictionnaires :

In [74]:
import requests
from scrapy import Selector

url = "https://www.imdb.com/chart/boxoffice"

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

r = requests.get(url, headers=headers)

response = Selector(text=r.text)

films = response.css("li[class='ipc-metadata-list-summary-item']")
classement = 0
dict_films = {}

for film in films:
    
    ### Création du dico d'un film
    dict_film = {}
    
    ### On crée le classement nous meme
    classement += 1
    dict_film['classement'] = classement
    
    ### On récupère le titre
    titre = film.css("h3[class='ipc-title__text']::text").get()
    dict_film['title'] = titre
    
    ### On récupère l'URL
    base_url="https://www.imdb.com"
    film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get('')
    url_film = base_url+film_relative_url
    dict_film['url'] = url_film
    
    
    ### On récupère les earnings
    detail_resp = requests.get(film_url, headers=headers, timeout=10)
    detail_sel = Selector(text=detail_resp.text)    
    earnings_raw = f= detail_sel.css(
        'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
        'span.ipc-metadata-list-item__list-content-item::text'
    ).get(default='?').strip()
    earnings = parse_earnings(earnings_raw)
    dict_film['earnings']  = earnings
    
    ### On récupère le rating
    rating = film.css("span.ipc-rating-star--rating::text").get()
    dict_film['rating'] = rating
    
    ### On récupère le nombre de votants
    nb_voteurs = parse_vote_count(film)
    dict_film['nb_voters'] = nb_voteurs
    
    dict_films[f'film number {classement}'] = dict_film

22000
22,000
1900
1,900
15000
15,000
1200
1,200
26000
26,000
79000
79,000
1100
1,100
128000
128,000
4800
4,800
56000
56,000


In [18]:
dict_films

{'film number 1': {'classement': 1,
  'title': None,
  'url': 'imdb.com/fr/title/tt8036976/?ref_=chtbo_i_1',
  'earnings': 9000000,
  'rating': '7,2',
  'nb_voters': 22000},
 'film number 2': {'classement': 2,
  'title': None,
  'url': 'imdb.com/fr/title/tt32306991/?ref_=chtbo_i_2',
  'earnings': 7000000,
  'rating': '7,6',
  'nb_voters': 1900},
 'film number 3': {'classement': 3,
  'title': None,
  'url': 'imdb.com/fr/title/tt27564844/?ref_=chtbo_i_3',
  'earnings': 6000000,
  'rating': '6,6',
  'nb_voters': 15000},
 'film number 4': {'classement': 4,
  'title': None,
  'url': 'imdb.com/fr/title/tt39216314/?ref_=chtbo_i_4',
  'earnings': 5700000,
  'rating': '8,5',
  'nb_voters': 1100},
 'film number 5': {'classement': 5,
  'title': None,
  'url': 'imdb.com/fr/title/tt31434030/?ref_=chtbo_i_5',
  'earnings': 4400000,
  'rating': '6,2',
  'nb_voters': 26000},
 'film number 6': {'classement': 6,
  'title': None,
  'url': 'imdb.com/fr/title/tt26443597/?ref_=chtbo_i_6',
  'earnings': 4000

## 🕷️ Étape 3 — Créer un crawler Scrapy

Maintenant que tes sélecteurs sont au point, transforme ton code en Spider Scrapy.

### 1. Crée une classe ImdbSpider(scrapy.Spider) :

In [75]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class imdb_spider(scrapy.Spider):
    # Name of your spider
    name = "imdb"

    # Url to start your spider from 
    start_urls = ["https://www.imdb.com/chart/boxoffice"]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        
        films = response.css("li[class='ipc-metadata-list-summary-item']")

        for film,i in zip(films,range(1,11)):
            ### Classement 
            classement = i
            
            ### Titre 
            titre = film.css("h3[class='ipc-title__text']::text").get()
            
            ### URL
            url_film_end = film.css("").get()
            url_film = "imdb.com" + url_film_end
            
            ### Earnings
            earnings_raw = film.css("").get()
            earnings = parse_earnings(earnings_raw)
            
            ### Ratings
            rating = film.css("").get()

            ### Nb votants
            nb_voteurs_raw = film.css("").getall()
            nb_voteurs = parse_vote_count(nb_voteurs_raw)

            yield {
                "ranking": classement,
                "title":   titre,
                "url":     url_film,
                "total_earnings": earnings,
                "rating":  rating,
                "nb_voters": nb_voteurs,
            }


# Name of the file where the results will be saved
filename = "imdb.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('01-Become_a_movie_director/'):
        os.remove('01-Become_a_movie_director/' + filename)

# Declare a new CrawlerProcess with some settings
## USER_AGENT => Simulates a browser on an OS
## LOG_LEVEL => Minimal Level of Log 
## FEEDS => Where the file will be stored 
## More info on built-in settings => https://docs.scrapy.org/en/latest/topics/settings.html?highlight=settings#settings
process = CrawlerProcess(settings = {
    "USER_AGENT": ("Chrome/140.0.0.0"),
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        '01-Become_a_movie_director/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(imdb_spider)
process.start()

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '01-Become_a_movie_director/'

### 2. Teste ton spider directement depuis le notebook :

In [ ]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class imdb_spider(scrapy.Spider):
    # Name of your spider
    name = "imdb"

    # Url to start your spider from 
    start_urls = ["https://www.imdb.com/chart/boxoffice"]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        
        films = response.css("li[class='ipc-metadata-list-summary-item']")

        for film,i in zip(films,range(1,11)):
            ### Classement 
            classement = i
            
            ### Titre 
            titre = film.css("h3[class='ipc-title__text']::text").get()
            
            ### URL
            url_film_end = film.css("a[href]::attr(href)").get()
            url_film = "imdb.com" + url_film_end
            
            ### Earnings
            earnings_raw = film.css("span[class='sc-382281d-2 bXJhOC']::text").get()
            earnings = parse_earnings(earnings_raw)
            
            ### Ratings
            rating = film.css("span[class='ipc-rating-star--rating']::text").get()

            ### Nb votants
            nb_voteurs_raw = film.css("span[class='ipc-rating-star--voteCount']::text").getall()
            nb_voteurs = parse_vote_count(nb_voteurs_raw)

            yield {
                "ranking": classement,
                "title":   titre,
                "url":     url_film,
                "total_earnings": earnings,
                "rating":  rating,
                "nb_voters": nb_voteurs,
            }


# Name of the file where the results will be saved
filename = "imdb.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('01-Become_a_movie_director/'):
        os.remove('01-Become_a_movie_director/' + filename)

# Declare a new CrawlerProcess with some settings
## USER_AGENT => Simulates a browser on an OS
## LOG_LEVEL => Minimal Level of Log 
## FEEDS => Where the file will be stored 
## More info on built-in settings => https://docs.scrapy.org/en/latest/topics/settings.html?highlight=settings#settings
process = CrawlerProcess(settings = {
    "USER_AGENT": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/124.0 Safari/124.0"),
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        '01-Become_a_movie_director/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(imdb_spider)
process.start()

2026-02-12 09:39:49 [scrapy.utils.log] INFO: Scrapy 2.13.3 started (bot: scrapybot)
2026-02-12 09:39:49 [scrapy.utils.log] INFO: Versions:
{'lxml': '5.3.0',
 'libxml2': '2.13.8',
 'cssselect': '1.2.0',
 'parsel': '1.8.1',
 'w3lib': '2.1.2',
 'Twisted': '24.11.0',
 'Python': '3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, '
           '16:37:03) [MSC v.1929 64 bit (AMD64)]',
 'pyOpenSSL': '25.1.0 (OpenSSL 3.5.3 16 Sep 2025)',
 'cryptography': '45.0.5',
 'Platform': 'Windows-11-10.0.26200-SP0'}
2026-02-12 09:39:49 [scrapy.addons] INFO: Enabled addons:
[]
2026-02-12 09:39:49 [scrapy.extensions.telnet] INFO: Telnet Password: c2821777f7c18491
2026-02-12 09:39:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2026-02-12 09:39:50 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/

RuntimeError: This event loop is already running

2026-02-12 09:39:51 [scrapy.core.engine] INFO: Spider opened
2026-02-12 09:39:51 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2026-02-12 09:39:51 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2026-02-12 09:39:52 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.imdb.com/chart/boxoffice/> (referer: None)
Traceback (most recent call last):
  File "c:\Users\dubos\miniforge3\envs\JedhaTraining\Lib\site-packages\scrapy\utils\defer.py", line 343, in iter_errback
    yield next(it)
          ~~~~^^^^
  File "c:\Users\dubos\miniforge3\envs\JedhaTraining\Lib\site-packages\scrapy\utils\python.py", line 369, in __next__
    return next(self.data)
  File "c:\Users\dubos\miniforge3\envs\JedhaTraining\Lib\site-packages\scrapy\utils\python.py", line 369, in __next__
    return next(self.data)
  File "c:\Users\dubos\miniforge3\envs\JedhaTraining\Lib\site-packages\scrapy\core\spidermw.py", line 16

: 

### 3. Maintenant tu peux exporter ton spider en un imdb.py et le déployer depuis ton terminal. 